In [44]:
# -*- coding: UTF-8 -*-

In [16]:
ls -l ../exp/tri2b/decode/scoring/

total 3784
-rw-rw-r-- 1 yh2901 yh2901 326206 Apr 17 01:41 10.tra
-rw-rw-r-- 1 yh2901 yh2901 322138 Apr 17 01:41 11.tra
-rw-rw-r-- 1 yh2901 yh2901 317665 Apr 17 01:41 12.tra
-rw-rw-r-- 1 yh2901 yh2901 313328 Apr 17 01:41 13.tra
-rw-rw-r-- 1 yh2901 yh2901 309390 Apr 17 01:41 14.tra
-rw-rw-r-- 1 yh2901 yh2901 305906 Apr 17 01:41 15.tra
-rw-rw-r-- 1 yh2901 yh2901 302505 Apr 17 01:41 16.tra
-rw-rw-r-- 1 yh2901 yh2901 299137 Apr 17 01:41 17.tra
-rw-rw-r-- 1 yh2901 yh2901 342203 Apr 17 01:41 7.tra
-rw-rw-r-- 1 yh2901 yh2901 336249 Apr 17 01:41 8.tra
-rw-rw-r-- 1 yh2901 yh2901 331092 Apr 17 01:41 9.tra
drwxrwxr-x 2 yh2901 yh2901   4096 Apr 17 01:41 log/
-rw-rw-r-- 1 yh2901 yh2901 341386 Apr 17 01:40 test_filt.txt


In [17]:
%%bash
less ../exp/tri2b/decode/scoring/17.tra | tail -2

NI45FBP_0101_2529230_2536840 9338 4946 4131 10104 11175 11147 6484 9157 9301 9311 8786 9616 8786 7663 11091 10794 8886 10938 10279 8771 
NI45FBP_0101_2536840_2547460 8449 959 3581 7900 7758 3975 10938 8850 3940 7652 8337 3581 2210 6444 2 5489 9209 8850 7663 


In [18]:
%%bash 
less ../exp/tri2b/decode/scoring/test_filt.txt | tail -2

UI29FAZ_0104_0975410_0977779 后 悔 是 OK SIL154
UI29FAZ_0104_0985128_0986293 ERR


In [19]:
%%bash
less ../data/lang/words.txt | head -5

<eps> 0
<oov> 1
A 2
ABACK 3
ABALONE 4


In [69]:
import os
from collections import defaultdict
import numpy as np

In [34]:
parent_path = os.path.split(os.getcwd())[0]
print ("parent path is {}".format(parent_path))

parent path is /home/yh2901/kaldi/egs/codeswitch_nomergesil


In [35]:
def get_file_list(path, extension):
    files = os.listdir(path)
    files = [f for f in files if f.endswith(extension)]
    print (files)
    return files

In [36]:
tra_path = parent_path + "/exp/tri2b/decode/scoring/"
tra_files = get_file_list(tra_path, ".tra") #need to get tri2b from terminal

['15.tra', '12.tra', '16.tra', '9.tra', '8.tra', '11.tra', '10.tra', '13.tra', '7.tra', '14.tra', '17.tra']


In [37]:
def gen_symbol_map():
    r"""
    Returns
    -------
    dic : defaultdict
      a dictionary with key as word symbol and value as the corresponding word
      e.g. '3': 'ABACK' 
    """
    dic = defaultdict(str)
    with open(parent_path + "/data/lang/words.txt", "r") as inputfile:
        for line in inputfile:
            word = line.split(" ")[0]
            symbol = line.strip("\n").split(" ")[1]
            dic[symbol] = word
    return dic
    

In [38]:
s2w = gen_symbol_map()

In [39]:
len(s2w)

11360

In [40]:
s2w['9338']

'因为'

In [41]:
def gen_ground_truth_map(model):
    r"""
    Returns
    -------
    dic : defaultdict
      a dictionary with utterance id as key and groud truth transcript as its value 
    """
    dic = defaultdict(list)
    with open(parent_path + "/exp/" + model + "/decode/scoring/test_filt.txt", "r") as inputfile:
        for line in inputfile:
            utterance_id = line.split(" ")[0]
            words = line.strip("\n").split(" ")[1:]
            dic[utterance_id] = words
    return dic


In [42]:
truth_map = gen_ground_truth_map("tri2b")

In [43]:
truth_map['NC01FBX_0101_0165090_0167860']

['不',
 '懂',
 'BUT',
 'OFFICIAL',
 'RESULT',
 '还没有',
 '出',
 'I',
 'THINK',
 '出',
 '了',
 '他们',
 '就会']

In [31]:
# compare two lists and find difference 
temp1 = ['那种', 'BUDGET', '现在', '还没有', '出来', '得', '出', '他们', '就会']# predicted
temp2 = ['不','懂','BUT','OFFICIAL','RESULT','还没有','出','I','THINK','出','了','他们','就会'] # truth
list(set(temp1) - set(temp2))

['那种', '现在', '出来', '得', 'BUDGET']

In [32]:
# compare two lists and find match
set(temp2).intersection(temp1)

{'他们', '出', '就会', '还没有'}

In [58]:
def isEnglish(word):
    from nltk.corpus import wordnet
    if word == "SIL1" or word == "SIL2":
        return -1
    if not wordnet.synsets(word):
        return 0
    else:
        return 1

In [130]:
def process_single_sentence(sen1, sen2):
    r"""sen2 as the ground truth, and sen1 as decoded label
    Returns
    -------
    number of match,
    number of english word match,
    number of chinese word match,
    number of silence phones match,
    total number of the truth sentence,
    total number of true words that are english,
    total number of true words that are chinese,
    total number of true words that are sil
    """
    intersection = set(sen2).intersection(sen1)
    # accuracy = len(intersection)/len(sen2)
    
    eng_inter = len([w for w in list(intersection) if isEnglish(w) == 1]) 
    # how many currect english word is captured 
    chi_inter = len([w for w in list(intersection) if isEnglish(w) == 0]) 
    # how many currect chinese word is capture
    sil_inter = len([w for w in list(intersection) if isEnglish(w) == -1]) 
    # how many engligh are there in ground truth
    total_true_eng = len([w for w in list(sen2) if isEnglish(w) == 1]) 
    # how many chinese are there in ground truth
    total_true_chi = len([w for w in list(sen2) if isEnglish(w) == 0]) 
    # how many SIL are there in ground truth
    total_true_sil = len([w for w in list(sen2) if isEnglish(w) == 0]) 
    
    return [len(intersection), eng_inter, chi_inter, sil_inter, len(sen2),total_true_eng, total_true_chi,total_true_sil ]
    

In [59]:
isEnglish("SIL1")

-1

In [124]:
a = [1,2,3]

In [125]:
a[-2]

2

In [128]:
def gen_file_stats(stacked):
    stacked = stacked[1:]
    sums = np.sum(stacked, axis=0)
    
    eng_WER = 1-(sums[1]/sums[-3])
    chi_WER = 1-(sums[2]/sums[-2])
    sil_WER = 1-(sums[3]/sums[-1])
    MER = 1-(sums[0]/sums[-4])
    return [chi_WER, eng_WER, sil_WER, MER]

In [60]:
[isEnglish(i) for i in temp1]

[0, 1, 0, 0, 0, 0, 0, 0, 0]

In [131]:
stacked = [0]*8
stats_stacked = [0]*4
counter = 0
for f in tra_files:
    with open(tra_path + f, "r") as inputfile:
        for line in inputfile:
                #print (line)
                symbols = line.replace("\n", "").split(" ")[1:]
                #print (symbols)
                decoded_words = [s2w[sym] for sym in symbols][:-1] # remove "" caused by newline 
                #print (decoded_words )
                true_words = truth_map[line.split(" ")[0]]
                #print (true_words)
                processed = process_single_sentence(decoded_words, true_words)
                stacked = np.vstack((stacked, processed))
    stacked = stacked[1:]
    stats = gen_file_stats(stacked)
    print (stats)
    stats_stacked = np.vstack((stats_stacked, stats))
    
# stats_stacked contains:    
#[chi_WER, eng_WER, sil_WER, MER]
stats_stacked = stats_stacked[1:]
means = np.mean(stats_stacked, axis=0)
chi_WER_mean = means[0]
eng_WER_mean = means[1]
sil_WER_mean = means[-2]
WER_mean = means[-1]
mins = np.min(stats_stacked, axis=0)
maxs = np.max(stats_stacked, axis=0)

print ("\n average: chi_WER {:.3f}, eng_WER: {:.3f} , mean_WER: {:.3f} ".format(chi_WER_mean, eng_WER_mean, WER_mean))
print ("\n min: chi_WER {:.3f}, eng_WER: {:.3f} , min_WER: {:.3f} ".format(mins[0], mins[1], mins[-1]))
print ("\n max: chi_WER {:.3f}, eng_WER: {:.3f} , max_WER: {:.3f} ".format(maxs[0], maxs[1], maxs[-1]))
                
            
                          
            
            
        

[0.66320695916099837, 0.66906878802058967, 1.0, 0.66452317908628589]
[0.65330099192368152, 0.65702653376386355, 1.0, 0.65413755057006251]
[0.65869437542353737, 0.66355694227769113, 1.0, 0.65978630232965485]
[0.65127232067224616, 0.65584704313028008, 1.0, 0.65229957539832684]
[0.64641656727745855, 0.65157159705711665, 1.0, 0.64757416648099642]
[0.64494806421152029, 0.64939381173056221, 1.0, 0.6459463247144559]
[0.64336476830425438, 0.64747422059944637, 1.0, 0.64428754197024385]
[0.6442559565159931, 0.64841555888410407, 1.0, 0.64519002609360587]
[0.64237864019181412, 0.64720751458916292, 1.0, 0.64346294622314715]
[0.64382822644701143, 0.64860234783097115, 1.0, 0.64490028442116198]
[0.64673655882178971, 0.6517569166092233, 1.0, 0.64786393387798002]

 average: chi_WER 0.649, eng_WER: 0.654 , mean_WER: 0.650 

 min: chi_WER 0.642, eng_WER: 0.647 , min_WER: 0.643 

 max: chi_WER 0.663, eng_WER: 0.669 , max_WER: 0.665 


In [132]:
sil_WER_mean

1.0